**Downgrade Datasets to 2.19.1**

In [2]:
# Cell 1
# Downgrade datasets to 2.19.1
!pip install datasets==2.19.1

# Restart the runtime after this step (Runtime → Restart runtime in Colab)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.3.1 which is incompatible.


**Upgrade Hugging Face Datasets**

In [1]:
# Cell 2
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.19.1
    Uninstalling datasets-2.19.1:
      Successfully uninstalled datasets-2.19.1


**Install Core Hugging Face Libraries**

In [2]:
# Cell 3
!pip install datasets==2.19.1 transformers accelerate evaluate tqdm

  Using cached datasets-2.19.1-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.19.1-py3-none-any.whl (542 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.4.1
    Uninstalling datasets-4.4.1:
      Successfully uninstalled datasets-4.4.1


**Install Bitsandbytes**

In [3]:
# Cell 4 — Install bitsandbytes
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.5 MB/s eta 0:00:00


**Upgrade Transformers + Accelerate**

In [4]:
# Cell 5 — Upgrade transformers + accelerate
!pip install -U transformers accelerate

**Import Core Libraries**

In [5]:
# Cell 5.5
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
from tqdm import tqdm

**Load OPT-350M Model + Tokenizer**

In [6]:
# Cell 6 — Load OPT-350M model + tokenizer

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "facebook/opt-350m"

# Configure 8-bit quantization for Colab efficiency
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

# Load the quantized model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

**Install PEFT**

In [7]:
# Cell 7
# Install PEFT
!pip install peft

from peft import LoraConfig, get_peft_model

# Define LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],  # typical for OPT
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap your quantized model with LoRA adapters
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 786,432 || all params: 331,982,848 || trainable%: 0.2369


**Load the Blended Skill Talk Dataset**

In [8]:
# Cell 8
from datasets import load_dataset

raw_datasets = load_dataset("blended_skill_talk")
print(raw_datasets)

Generating train split:   0%|          | 0/4819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/980 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 4819
    })
    validation: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 980
    })
})


**Tokenize Blended Skill Talk Dataset**

In [9]:
# Cell 9 — Tokenize Blended Skill Talk dataset

from transformers import DataCollatorForLanguageModeling

# Function to join dialogue turns into a single string
def preprocess_function(examples):
    # Use free_messages if available, otherwise guided_messages
    texts = []
    for fm, gm in zip(examples.get("free_messages", []), examples.get("guided_messages", [])):
        if fm:
            texts.append(" ".join(fm))
        elif gm:
            texts.append(" ".join(gm))
        else:
            texts.append("")
    return tokenizer(texts, truncation=True, padding="max_length", max_length=128)

# Apply preprocessing
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

print(tokenized_datasets)

Map:   0%|          | 0/4819 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/980 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4819
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 980
    })
})


**Full Training Setup with Per-epoch Metrics**

In [10]:
# Cell 10 — Full training setup with per-epoch metrics

import math
from transformers import Trainer, TrainingArguments

# Training arguments (no evaluation_strategy/logging_strategy here)
training_args = TrainingArguments(
    output_dir="./opt350m-bst",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    fp16=True,
    save_steps=500
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Run training + evaluation per epoch
num_epochs = training_args.num_train_epochs
for epoch in range(num_epochs):
    print(f"\n=== Starting Epoch {epoch+1} ===")
    trainer.train(resume_from_checkpoint=None)

    # Validation metrics
    eval_results = trainer.evaluate()
    val_loss = eval_results["eval_loss"]
    val_ppl = math.exp(val_loss)

    # Training metrics: average loss from last epoch
    train_logs = [log for log in trainer.state.log_history if "loss" in log]
    train_loss = train_logs[-1]["loss"] if train_logs else None
    train_ppl = math.exp(train_loss) if train_loss else None

    print(f"\nEpoch {epoch+1} Summary:")
    print(f"  Train Loss: {train_loss:.4f}" if train_loss is not None else "  Train Loss: N/A")
    print(f"  Train PPL : {train_ppl:.4f}" if train_ppl is not None else "  Train PPL : N/A")
    print(f"  Val Loss  : {val_loss:.4f}")
    print(f"  Val PPL   : {val_ppl:.4f}")

/tmp/ipython-input-2626675770.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



=== Starting Epoch 1 ===


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


...

/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q


Epoch 2 Summary:
  Train Loss: 3.2317
  Train PPL : 25.3227
  Val Loss  : 3.2324
  Val PPL   : 25.3409

=== Starting Epoch 3 ===


Step,Training Loss
50,3.167100
100,3.190300
150,3.157000
200,3.272200
250,3.233500
300,3.200900
350,3.178500
400,3.235600
450,3.282200
500,3.181500


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q


Epoch 3 Summary:
  Train Loss: 3.2076
  Train PPL : 24.7197
  Val Loss  : 3.2237
  Val PPL   : 25.1219
